In [1]:
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import tensorflow as tf

import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure

import numpy as np
import matplotlib.pyplot as plt

train_data, test_data = tfds.load('stl10', 
                        split=['train', 'test'],
                        as_supervised=True, 
                        shuffle_files=True,
                        data_dir=".")

In [2]:
# hog can't accept our TF dataset, so converting to numpy arr
np_imgs = tfds.as_numpy(train_data)

# Using list comprehension to process each image with hog, then convert to np array
hog_imgs = np.array([(hog(x, multichannel=True), l) for x, l in np_imgs])

# Cannot convert this data directly to a tensor due to its shape caused by input and label being together, so I separate them out
input = hog_imgs[0:, 0]
labels = hog_imgs[0:, 1]

# TF didn't like the numpy types, so I convert them to a list and then convert the list back to a tensor for each one
input_tensor = tf.convert_to_tensor(list(input))
labels_tensor = tf.convert_to_tensor(list(labels))

# Combine data back into a dataset
hog_train_data = tf.data.Dataset.from_tensor_slices((input_tensor, labels_tensor))

In [3]:
BATCH_SIZE = 25
#STEPS_PER_EPOCH = len(train_data) / BATCH_SIZE 

def process(img, label):
  return img, tf.one_hot(label, depth=10, dtype=tf.int8)

processed_dataset = hog_train_data.map(process).shuffle(64).batch(BATCH_SIZE)

In [4]:
label_map = ['plane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'primate', 'boat', 'truck']

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(32, 3, input_shape=[8100, 1]),
  tf.keras.layers.Conv1D(32, 3, strides=2),
  tf.keras.layers.Conv1D(64, 3, input_shape=[8100, 1]),
  tf.keras.layers.Conv1D(64, 3, strides=2),
  tf.keras.layers.Conv1D(128, 3),
  tf.keras.layers.Conv1D(64, 3),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Flatten(data_format="channels_last"),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

#model.summary()

In [9]:

#print(processed_dataset[0])
#processed_dataset = np.expand_dims(processed_dataset, axis=1)
history = model.fit(processed_dataset, epochs=10)

AxisError: axis 1 is out of bounds for array of dimension 1